In [1]:
import spacy
import pandas as pd

data = pd.read_csv("./datasets/mensagens.csv")

A base de dados foi gerada pela query:

```
union customEvents
| where name == "MessageIn" and customDimensions ["IntentName"] == "unknown"
| order by timestamp
| project Messages = parse_json(tostring(customDimensions["Message"])).text
| where Messages  != ""
```

Ou seja, todas as frases de intenção desconhecida

O algoritmo de frequencia considera o tipo sintatico da palavra, além do que desconsidera palavras irrelevantes e remove plurais.

In [19]:
nlp = spacy.load("pt_core_news_sm")
    
wordFrequency = {}
def computeFrequency(wordFrequency, nlp, data, wordType = ['NOUN']):
    for string in data.Messages:
        string = string.lower()
        doc = nlp(string)
        for token in doc:
            if token.is_stop:
                continue
            if token.pos_ in wordType:
                text = token.text
                if text[-1] == "s":
                    text = text[:-1]
                if text in wordFrequency.keys():
                    wordFrequency[text]+=1
                else:
                    wordFrequency[text]=1


In [22]:
computeFrequency(wordFrequency,nlp,data.head(5000))

### Número Máximo de Palavras

Para alterar o número máximo de palavras altere a variável a seguir:

In [23]:
max_words = 200

### Núvem sem aplicar limpeza 

A primeira demonstração é baseada no texto todo, sem aplicar nenhum modelo de NLP para identificar os tipos sintáticos. O resultado foi bastante poluido, além de encontrar termos concatenados.

In [24]:
from wordcloud import WordCloud

text = ''
for phrase in data.Messages:
    text += phrase + "\n"
wordcloud = WordCloud().generate(text)

import matplotlib.pyplot as plt

wordcloud = WordCloud(background_color="white",max_words=max_words,max_font_size=40, relative_scaling=.5).generate(text)
plt.figure(figsize=(100,100), dpi = 200)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

### Núvem de substantivos

In [26]:
wordcloud = WordCloud(background_color="white",max_words=max_words,max_font_size=50, relative_scaling=.5).generate_from_frequencies(wordFrequency)
plt.figure(figsize=(100,100), dpi = 200)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

### Núvem de Verbos

In [27]:
wordFrequencyTest = {}
computeFrequency(wordFrequencyTest,nlp,data.head(5000), ['VERB'])

In [21]:
wordcloud = WordCloud(background_color="white",max_words=max_words,max_font_size=40, relative_scaling=.5).generate_from_frequencies(wordFrequencyTest)
plt.figure(figsize=(100,100), dpi = 200)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()